# A Bayesian model for a coin toss

**Goal:** In this notebook you will learn how you can calculate the postirior distribution for the parameter $\theta$ as well as the predictvie distribution for the outcome of a bernoulli experiment. You will use the brute force approach and pick different prior distributions for the parameter $\theta$.
Note that you will use discrete values for the prior and postirior of $\theta$ here. You will work with sums in this notebook to approximate the integrals.


**Dataset:** You work with the observed values of coin tosses, 1 for head and 0 for tail

**Content:**
* Define a uniform prior for $\theta$
* Evaluate the joined likelihood and the unnormalized posterior at one specific $\theta$
* Calculate the joined likelihood, the unnormalized posterior and the normalized posterior for a range of  $\theta$
* Calculate and plot the prior predictive distribution and the posterior predictive distribution




In [0]:
!pip install tensorflow==2.1.0

In [0]:
!pip install tensorflow_probability==0.8.0

#### Imports

In [0]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp

plt.style.use('default') 
%matplotlib inline
tfd = tfp.distributions
print("TFP Version", tfp.__version__)
print("TF  Version",tf.__version__)

#### Create train data
In the next cell we set the outcome of three coin tosses to one, meaning we get three times head.

In [0]:
obs_data=np.repeat(1,3)


## Maximum likelihood approach to fit the Bernoulli model
Because we have a binary outcome (heads or tails) we need a Bernoulli model to describe the data. A Bernoulli model has only one parameter $\theta$ which is the probability to get outcome one (head). In the cell below you use the maximum liklihood approach to estimate the parameter $\theta$ and the standart deviation.

In [0]:
### ML
est_theta_ml=np.mean(obs_data)
print("ML theta",est_theta_ml)
sd_est_theta_ml = est_theta_ml * (1. - est_theta_ml)
print("sd theta",sd_est_theta_ml)

## Bayes approach to fit the Bernoulli model
For the Bayes approach we fist need to define a prior distribution for the patameter $\theta$ of the Bernoulli distribution. We evaluate the distributions at discrete points in the range 0.05 to 0.95. You use a uniform prior where every theta has the same probability. 

In [0]:
theta=np.arange(0.05,1,0.05)
print(theta)
prior = 1/len(theta) #The normalizing constant of the prior

Let's evaluate the joined likelihood and the unnormalized posterior at one specific $\theta = 0.5$

In [0]:
dist = tfp.distributions.Bernoulli(probs=0.5) #one specific theta
print(np.prod(dist.prob(obs_data))) #joint likelihood
print(np.prod(dist.prob(obs_data))*prior) #unnormalized posterior

Repeating the steps above for all thetas from the range of 0.05 until 0.95.

In [0]:
res = np.zeros((len(theta),5))
for i in range(0,len(theta)):
  dist = tfp.distributions.Bernoulli(probs=theta[i])   
  res[i,0:4]=np.array((theta[i],np.prod(dist.prob(obs_data)),prior,np.prod(dist.prob(obs_data))*prior))

Note that we need to normalize the posterior, so that it sums up to one.

In [0]:
import pandas as pd
res=pd.DataFrame(res,columns=["theta","jointlik","prior","unnorm_post","post"])
res["post"]=res["unnorm_post"]/np.sum(res["unnorm_post"])
res

### Posterior and prior for $\theta$

Let's plot the prior and the posterior for $\theta$

In [0]:
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.stem(res["theta"],res["prior"],use_line_collection=True)
plt.xlabel("theta")
plt.ylabel("probability")
plt.ylim([0,0.5])
plt.title("prior distribution")
plt.subplot(1,2,2)
plt.stem(res["theta"],res["post"],use_line_collection=True)
plt.ylim([0,0.5])
plt.xlabel("theta")
plt.ylabel("probability")
plt.title("posterior distribution")

### Posterior and prior for head

The predictive distribution is given by

$
p(y|D)=\sum_i p(y|\theta_i)p(\theta_i|D)
$

For the probability of head, we have $\theta_i = p(y=1|\theta_i)$. For $p(\theta_i|D)$, we first use the posterior distribution resulting in

$
  p(y=1|D) = \sum_i \theta_i p(\theta_i|D)
$

In [0]:
py1_post = np.sum((res["theta"])*res["post"])
py0_post = 1.0 - py1_post
py0_post, py1_post

Similar, you get for the prior distributions: 

In [0]:
py1_prior = np.sum((res["theta"])*res["prior"]) 
py0_prior = 1 - py1_prior
py0_prior, py1_prior

We plot this togther with the figure above.  

In [0]:
plt.figure(figsize=(16,12))

plt.subplot(2,2,1)
plt.stem(res["theta"],res["prior"],use_line_collection=True)
plt.xlabel("theta")
plt.ylabel("probability")
plt.ylim([0,0.5])
plt.title("prior distribution for theta")
plt.subplot(2,2,2)
plt.stem(res["theta"],res["post"],use_line_collection=True)
plt.ylim([0,0.5])
plt.xlabel("theta")
plt.ylabel("probability")
plt.title("posterior distribution for theta")
plt.savefig("test.pdf")

plt.subplot(2,2,3)
plt.stem([0,1],[py0_prior,py1_prior],use_line_collection=True)
plt.xlabel("Y")
plt.ylabel("P(y)")
plt.ylim([0,1])
plt.title("prior predictive distribution")
plt.subplot(2,2,4)
plt.stem([0,1],[py0_post,py1_post],use_line_collection=True)
plt.ylim([0,1])
plt.xlabel("Y")
plt.ylabel("P(y)")
plt.title("posterior predictive distribution")
plt.show()

### Posterior and the predictive distribution for different observed data

*Exercise: Let's assume fist you observed 40 times head and then you observed 11 times head and 9 times tail. How does the posterior and the predictive distribution look, for these two cases?*

In [0]:
# Write your code here

### Different prior

*Exercise 2: Let's now repeat the experiment with a non-uniform distributed prior. You will use a halfcircle as prior. With this prior calculate again the posterior when you fist observe 40 times head and then you observed 11 times head and 9 times tail. How does the posterior and the predictive distribution look with the new prior?*

In [0]:
prior=np.sqrt(np.square(0.5)-np.square(theta-0.5))-0.2
prior=prior/np.sum(prior)#normalzation

In [0]:
plt.stem(theta,prior,use_line_collection=True)
plt.xlim([0,1])

In [0]:
# Write your code here